#### Zomato Scrapper 1

In [31]:
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
import numpy as np

In [2]:
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
# Initialise a chrome webdriver instance with ChromeDriverManager. 

In [17]:
restaurants_dict = {"name": [], "location": [],"price_for_two": [], 
                    "cuisines": [], "rating": [], "restaurant_link": []}

location = ["", "whitefield-restaurants", "marathahalli-restaurants"]

driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

for k in location:
    driver.get("https://www.zomato.com/bangalore/"+k)
    driver.maximize_window()

    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(40)

    page_data = BeautifulSoup(driver.page_source)

    for i in page_data.find_all("a", class_="sc-bvCTgw dIvKTC"):
        restaurants_dict["restaurant_link"].append("https://www.zomato.com" + i["href"])
        try:
            restaurants_dict["name"].append(i.find("h4", class_="sc-1hp8d8a-0 sc-ertOQY hArupt").text)
        except(AttributeError):
            restaurants_dict["name"].append(i.find("h4", class_="sc-1hp8d8a-0 sc-cgThhu dFwWJC").text)
        restaurants_dict["rating"].append(i.find("div", class_="sc-1q7bklc-1 cILgox").text)
        try:
            restaurants_dict["cuisines"].append(i.find("p", class_="sc-1hez2tp-0 sc-iEPtyo bHxcOC").text)
        except(AttributeError):
            restaurants_dict["cuisines"].append(i.find("p", class_="sc-1hez2tp-0 sc-LAuEU fDjWNG").text)
        try:
            restaurants_dict["location"].append(i.find("p", class_="sc-1hez2tp-0 sc-gUlUPW icrStE").text.strip())
        except(AttributeError):
            restaurants_dict["location"].append(i.find("p", class_="sc-1hez2tp-0 sc-gkfylT cOAsXn").text.strip())
        try:
            restaurants_dict["price_for_two"].append(i.find("p", class_="sc-1hez2tp-0 sc-iEPtyo dJHUYi").text.split()[0].replace("₹", "").replace(",", ""))
        except(AttributeError):
            restaurants_dict["price_for_two"].append(i.find("p", class_="sc-1hez2tp-0 sc-LAuEU dHtbEm").text.split()[0].replace("₹", "").replace(",", ""))

# Scrape restaurants data from zomato for specified location and store it in the restaurants dictionary
# using chrome drive and beautifulsoup.

In [18]:
restaurants_df = pd.DataFrame(restaurants_dict)
# create a restaurants dataframe using the dictionary.

In [68]:
restaurants_df.drop_duplicates(inplace=True)
# Drop duplicate values from the dataframe.

In [83]:
restaurants_df = restaurants_df.head(1000)
restaurants_df.reset_index(inplace=True)
restaurants_df.drop("index", axis=1, inplace=True)
# Extract 1000 restaurants from the dataframe.

In [85]:
restaurants_df.isna().sum()
# Check missing values count for each columns.

name               0
location           0
price_for_two      0
cuisines           0
rating             0
restaurant_link    0
dtype: int64

In [86]:
restaurants_df.to_csv("restaurants_data.csv")
# Export the restaurants dataframe as .csv file.

#### Zomato Scrapper 2

In [94]:
restaurant_details_dict = {"name": [], "latitude": [], "longitude": [], 
    "delivery_review_number": [], "location": []}

In [95]:
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

for i in restaurants_df["restaurant_link"]:
    driver.get(i)
    time.sleep(3)
    current_page_data = BeautifulSoup(driver.page_source)
    restaurant_details_dict["name"].append(current_page_data.find("h1", class_="sc-7kepeu-0 sc-iSDuPN fwzNdh").text)
    restaurant_details_dict["location"].append(current_page_data.find("a", class_="sc-clNaTc vNCcy").text) 
    restaurant_details_dict["delivery_review_number"].append(current_page_data.find_all("div", class_="sc-1q7bklc-8 kEgyiI")[1].text) 
    try:
        location = current_page_data.find("a", class_="sc-cIwbeI hdnZmZ")["href"]
        splits = location.split("=")
        latitude, longitude = splits[-1].split(",")
        restaurant_details_dict["latitude"].append(latitude)
        restaurant_details_dict["longitude"].append(longitude)
    except:
        restaurant_details_dict["latitude"].append(np.nan)
        restaurant_details_dict["longitude"].append(np.nan)

# Scrape restaurants details data from zomato for scrapped restaurants and store it in the restaurant details dictionary 
# using chrome drive and beautifulsoup.

In [97]:
restaurant_details_df = pd.DataFrame(restaurant_details_dict)
# create a restaurant details dataframe using the dictionary.

In [100]:
restaurant_details_df.to_csv("restaurants_details.csv")
# Export the restaurant details dataframe as .csv file.